In [9]:
# Setup:

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
# SELU activation function example:

from scipy.special import erfc

# Alpha and scale to self normalize with mean 0 and std dev of 1
alpha_0_1 = -np.sqrt(2/np.pi) / (erfc(1/np.sqrt(2))* np.exp(1/2) - 1)
scale_0_1 = (1 - erfc(1 / np.sqrt(2)) * np.sqrt(np.e)) * np.sqrt(2 * np.pi) * (2 * erfc(np.sqrt(2)) *np.e**2 + np.pi*erfc(1/np.sqrt(2)) **2*np.e - 2*(2+np.pi)*erfc(1/np.sqrt(2))*np.sqrt(np.e)+ np.pi+2)**(-1/2)

In [7]:
def elu(z, alpha=1):
    return np.where(z < 0, alpha * (np.exp(z) - 1), z)

In [4]:
def selu(z, scale=scale_0_1, alpha = alpha_0_1):
    return scale * elu(z, alpha)

In [16]:
np.random.seed(42)
Z = np.random.normal(size=(500, 100)) # standardized inputs (normal)

for layer in range(100):
    W = np.random.normal(size=(100, 100), scale = np.sqrt(1/100)) # LeCun initialization (shouldn't there be a method for that?)
    Z = selu(np.dot(Z, W))
    means = np.mean(Z, axis = 0).mean()
    stds = np.std(Z, axis = 0).mean()
    if layer % 10 == 0:
        print("Layer {}: mean {:.2f}, std deviation {:.2f}".format(layer, means, stds))

Layer 0: mean -0.00, std deviation 1.00
Layer 10: mean 0.00, std deviation 0.97
Layer 20: mean -0.01, std deviation 0.96
Layer 30: mean 0.01, std deviation 0.97
Layer 40: mean -0.03, std deviation 0.94
Layer 50: mean 0.04, std deviation 0.98
Layer 60: mean 0.05, std deviation 0.96
Layer 70: mean 0.00, std deviation 0.90
Layer 80: mean -0.04, std deviation 0.90
Layer 90: mean -0.01, std deviation 0.90


In [17]:
# Using SELU:

keras.layers.Dense(10, activation="selu",
                  kernel_initializer="lecun_normal")

np.random.seed(42)
tf.random.set_seed(42)


In [19]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28,28]))
model.add(keras.layers.Dense(300, activation="selu",
                            kernel_initializer="lecun_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                kernel_initializer="lecun_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [25]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

In [21]:


(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]



In [23]:
# Now let's trian the model, remembering to scale the inputs to a mean of 0 and std dev of 1:

pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [26]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                   validation_data=(X_valid_scaled, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/5
55000/55000 [==============================] - 54s 976us/sample - loss: 1.1867 - accuracy: 0.5625 - val_loss: 0.7696 - val_accuracy: 0.7252
Epoch 2/5
55000/55000 [==============================] - 42s 766us/sample - loss: 0.6680 - accuracy: 0.7637 - val_loss: 0.6368 - val_accuracy: 0.7628
Epoch 3/5
55000/55000 [==============================] - 41s 753us/sample - loss: 0.5605 - accuracy: 0.8034 - val_loss: 0.5448 - val_accuracy: 0.8136
Epoch 4/5
55000/55000 [==============================] - 44s 791us/sample - loss: 0.5115 - accuracy: 0.8241 - val_loss: 0.4939 - val_accuracy: 0.8280
Epoch 5/5
55000/55000 [==============================] - 43s 783us/sample - loss: 0.4760 - accuracy: 0.8363 - val_loss: 0.4912 - val_accuracy: 0.8338


In [27]:
# ^ This shows a model that's suffering from vanishing/exploding gradients.

# Batch normalization:

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_203 (Dense)            (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_204 (Dense)            (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_205 (Dense)            (None, 10)               

In [29]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [30]:
bn1.updates

[<tf.Operation 'cond/Identity' type=Identity>,
 <tf.Operation 'cond_1/Identity' type=Identity>]

In [32]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.SGD(lr=1e-3),
             metrics=["accuracy"])

In [33]:
history = model.fit(X_train, y_train, epochs=10,
                   validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 10s 181us/sample - loss: 0.8624 - accuracy: 0.7074 - val_loss: 0.5546 - val_accuracy: 0.8080
Epoch 2/10
55000/55000 [==============================] - 10s 174us/sample - loss: 0.5752 - accuracy: 0.7997 - val_loss: 0.4771 - val_accuracy: 0.8326
Epoch 3/10
55000/55000 [==============================] - 9s 165us/sample - loss: 0.5177 - accuracy: 0.8165 - val_loss: 0.4422 - val_accuracy: 0.8414
Epoch 4/10
55000/55000 [==============================] - 9s 159us/sample - loss: 0.4827 - accuracy: 0.8303 - val_loss: 0.4223 - val_accuracy: 0.8496
Epoch 5/10
55000/55000 [==============================] - 9s 158us/sample - loss: 0.4604 - accuracy: 0.8375 - val_loss: 0.4069 - val_accuracy: 0.8554
Epoch 6/10
55000/55000 [==============================] - 9s 158us/sample - loss: 0.4389 - accuracy: 0.8429 - val_loss: 0.3938 - val_accuracy: 0.8578
Epoch 7/10
55000/55000 [=========================

In [35]:
# In some cases, it's better to apply batch normalization before the activation function:
#   Notice the addition of an Activation layer after BN and the 'use_bias=False' swapped in the Dense hidden layers (BN has bias it adds so doing it twice is redundant-ish):

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [36]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.SGD(lr=1e-3),
             metrics=["accuracy"])

In [37]:
history = model.fit(X_train, y_train, epochs=10,
                   validation_data=(X_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 10s 175us/sample - loss: 1.0651 - accuracy: 0.6690 - val_loss: 0.6814 - val_accuracy: 0.7914
Epoch 2/10
55000/55000 [==============================] - 9s 172us/sample - loss: 0.6940 - accuracy: 0.7778 - val_loss: 0.5620 - val_accuracy: 0.8206
Epoch 3/10
55000/55000 [==============================] - 9s 168us/sample - loss: 0.6022 - accuracy: 0.8010 - val_loss: 0.5021 - val_accuracy: 0.8378
Epoch 4/10
55000/55000 [==============================] - 9s 161us/sample - loss: 0.5508 - accuracy: 0.8153 - val_loss: 0.4687 - val_accuracy: 0.8456
Epoch 5/10
55000/55000 [==============================] - 9s 158us/sample - loss: 0.5186 - accuracy: 0.8239 - val_loss: 0.4440 - val_accuracy: 0.8518
Epoch 6/10
55000/55000 [==============================] - 9s 164us/sample - loss: 0.4948 - accuracy: 0.8313 - val_loss: 0.4264 - val_accuracy: 0.8560
Epoch 7/10
55000/55000 [==========================

In [40]:
# Performance Scheduling

tf.random.set_seed(42)
np.random.seed(42)

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience=5)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])

optimizer = keras.optimizers.SGD(lr=0.02, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 25
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                   validation_data = (X_valid_scaled, y_valid),
                   callbacks=[lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/25
55000/55000 [==============================] - 7s 123us/sample - loss: 0.6095 - accuracy: 0.8021 - val_loss: 0.5357 - val_accuracy: 0.8206
Epoch 2/25
55000/55000 [==============================] - 6s 110us/sample - loss: 0.5148 - accuracy: 0.8369 - val_loss: 0.5767 - val_accuracy: 0.8284
Epoch 3/25
55000/55000 [==============================] - 6s 111us/sample - loss: 0.5219 - accuracy: 0.8420 - val_loss: 0.5697 - val_accuracy: 0.8416
Epoch 4/25
55000/55000 [==============================] - 7s 121us/sample - loss: 0.5265 - accuracy: 0.8446 - val_loss: 0.5637 - val_accuracy: 0.8384
Epoch 5/25
55000/55000 [==============================] - 6s 113us/sample - loss: 0.5371 - accuracy: 0.8471 - val_loss: 0.8142 - val_accuracy: 0.8372
Epoch 6/25
55000/55000 [==============================] - 6s 110us/sample - loss: 0.5128 - accuracy: 0.8543 - val_loss: 0.6704 - val_accuracy: 0.8330
Epoch 7/25
55000/55000 [===========================

In [45]:
class OneCycleScheduler(keras.callbacks.Callback):
    def __init__(self, iterations, max_rate, start_rate=None,
                 last_iterations=None, last_rate=None):
        self.iterations = iterations
        self.max_rate = max_rate
        self.start_rate = start_rate or max_rate / 10
        self.last_iterations = last_iterations or iterations // 10 + 1
        self.half_iteration = (iterations - self.last_iterations) // 2
        self.last_rate = last_rate or self.start_rate / 1000
        self.iteration = 0
    def _interpolate(self, iter1, iter2, rate1, rate2):
        return ((rate2 - rate1) * (self.iteration - iter1)
                / (iter2 - iter1) + rate1)
    def on_batch_begin(self, batch, logs):
        if self.iteration < self.half_iteration:
            rate = self._interpolate(0, self.half_iteration, self.start_rate, self.max_rate)
        elif self.iteration < 2 * self.half_iteration:
            rate = self._interpolate(self.half_iteration, 2 * self.half_iteration,
                                     self.max_rate, self.start_rate)
        else:
            rate = self._interpolate(2 * self.half_iteration, self.iterations,
                                     self.start_rate, self.last_rate)
            rate = max(rate, self.last_rate)
        self.iteration += 1
        K.set_value(self.model.optimizer.lr, rate)

In [47]:
batch_size = 128
n_epochs = 25
# Not sure why we're using floor division here, but I'll go with it:
onecycle = OneCycleScheduler(len(X_train) // batch_size * n_epochs, max_rate=0.05)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs, batch_size = batch_size,
                   validation_data=(X_valid_scaled, y_valid),
                   callbacks=[onecycle])

Train on 55000 samples, validate on 5000 samples
Epoch 1/25
55000/55000 [==============================] - 3s 51us/sample - loss: 0.0506 - accuracy: 0.9830 - val_loss: 0.5244 - val_accuracy: 0.8904
Epoch 2/25
55000/55000 [==============================] - 3s 50us/sample - loss: 0.0559 - accuracy: 0.9805 - val_loss: 0.5370 - val_accuracy: 0.8864
Epoch 3/25
55000/55000 [==============================] - 3s 48us/sample - loss: 0.0623 - accuracy: 0.9772 - val_loss: 0.5663 - val_accuracy: 0.8898
Epoch 4/25
55000/55000 [==============================] - 3s 48us/sample - loss: 0.0749 - accuracy: 0.9717 - val_loss: 0.5777 - val_accuracy: 0.8866
Epoch 5/25
55000/55000 [==============================] - 3s 46us/sample - loss: 0.0930 - accuracy: 0.9650 - val_loss: 0.5979 - val_accuracy: 0.8866
Epoch 6/25
55000/55000 [==============================] - 3s 48us/sample - loss: 0.1027 - accuracy: 0.9613 - val_loss: 0.5885 - val_accuracy: 0.8864
Epoch 7/25
55000/55000 [==============================] -

In [ ]:
# The validation loss (0.773) and accuracy discrepencies suggest that the model is fairly significantly overfitting the data.
#   I'mma fix this with some better regularization techniques:

